# Lambda, the Pynultimate Imperative

#### Version 2
#### Brian Beckman
#### 12 Nov 2022
#### [Creative Commons Attribution 4.0 International Public License](https://creativecommons.org/licenses/by/4.0/legalcode)

# Introduction

In a classic paper, [_Lambda, the Ultimate Imperative_](https://www.researchgate.net/publication/37596655_Lambda_The_Ultimate_Imperative), Steele and Sussman show how to model most imperative constructs with just _lambda:_

> We demonstrate how to model the following common [imperative] programming constructs in ... an applicative-order language similar to LISP: Simple Recursion, Iteration, Compound Statements and Expressions, GO TO and Assignment, Continuation-Passing, Escape Expressions, Fluid Variables, Call by Name, Call by Need, and Call by Reference. The models require only (possibly self-referent) lambda application, conditionals, and (rarely) assignment.

It's useful to recap this paper in Python, which has most of the listed imperative constructs. Imagine compiling Python into an intermediate language in which the semantics, even those with side-effects, are laid bare as trees of $\lambda$ expressions. In such a representation, optimizations are 
1. easy to write as tree-to-tree transforms
1. easy to extend via just function composition (even Kleisli-monadic)
2. independent of surface syntax, thus easy to share with other imperative languages like Fortran, C, Java
3. independent of back ends, thus easy to run interactively; or to translate into LLVM, x86, ARM64, C, for execution; or to transpile into other surface languages

The use-cases above are similar to those for a SQL algebraizer. Many SQL implementations 
1. translate the surface language into bare-bones expressions in a closed relational algebra, free of original syntax
2. run the algebraic expressions through symbolic optimizers, which often rearrange the expressions completely
2. incrementally extend the system by composing new optimizations
3. translate optimized expressions into commands for local and distributed servers

We follow the paper more-or-less directly, with refernce to [SICP](https://sarabander.github.io/sicp/).

## _Schemulation:_ Python Semantics in Python

Ideally, we'd compile Python into Scheme or Clojure or Common Lisp, then write transformations, translations, interpreters, debuggers, etc. in Scheme or Clojure or Common Lisp. However, to maintain a convenient notebook structure and to avoid creeping dependencies, we'll just model Python imperatives in a Scheme-like applicative-order $\lambda$-calculus embedded in basic Python.

Some people may find this weird. Why not just implement Scheme in Python instead of emulating Scheme in Python? It's an engineering judgment call. Most authors of compilers and interpreters spend undue time on syntax before even getting to semantics. Compiler textbooks tell you to do this! Semantics becomes a hidden "implementation detail," but then developers forgive themselves for making an ungodly mess of it. We prefer the other way around. Get the semantics clean, composable, extendable, maintainable, and correct, _first_, then spend time on syntax, if you even care any more. Or just let someone else do it. From the development point of view, syntax is a distraction. It's been solved for decades, but it makes young developers and professors feel powerful. The decision to spend time on syntax before semantics is not engineering, it's self-indulgence.

## Orthogonality as a Design Principle

We prefer designs that minimize cross-talk. Each facility -- transformation layer or module -- should have the least possible dependency on other facilities. For example, tranformations that affect control flow need not necessarily depend on transformations that affect numerics.

Contrast to a braided design, where each facility explicitly accounts for every other. Such a design is not usually created on purpose: it accretes as semantics is bodged on to syntax trees. Correcting such a design requires total refactoring after-the-fact. It's much easier to get it right before-the-fact.

# Environment and Frame<a id="environment"></a>

We find it necessary to model Scheme's environments and frames explicitly. We tried multiple short-cut alternatives and found that none of them compose well.

[SICP 3.2](https://sarabander.github.io/sicp/html/3_002e2.xhtml#g_t3_002e2) has some apparent contradictions in the definition of environment and frame. It says that "an environment is a sequence of frames," but the rest of the text and figures clearly imply that an environment has just one frame.

The best resolution appears to be:

> An ___environment___ is a frame $\phi$ and a pointer $\pi$ to an enclosing environment. A ___frame___ is a mathematical function from variable names to values; no variable name may appear more than once in a frame.

We note in passing that this works only for a single thread. [Clojure, for instance, solves that problem with _Vars_](https://clojure.org/reference/vars).

## Greek and ALL CAPS<a id="greek"></a>

$\pi$ for an enclosing environment is a nice pun: it evokes $\pi\eta\rho\iota$, a Greek prefix meaning "surrounding."

We use Greek letters for system attributes. User code should avoid Greek to avoid clobbering system attributes.

We use ALL CAPS for system functions that implement Schemulator.

## Bindings

> A ___binding___ is an association from a variable name to a value, that is, an entry in a frame.

We might model a binding as a pair, or as a row in a table, an element of a relation (subset of a Cartesian product), an element of a Python dictionary, or as an attribute of a Python object. We prefer the attribute model because it affords _dot_ notation for lookup, that is, `o.foo` rather than the dictionary's syntax `o['foo']`. Thanks to [divs1210](https://gist.github.com/divs1210?page=3) for this idea of modeling frames as dummy lambda functions, namely `lambda: None`, and assigning attributes to them.

If the definitions above are acceptable, the apparent contradiction in SICP is resolved. SICP says that an environment _is_ a sequence of frames. Rather, I'd say that any environment _implies_ a virtual sequence of frames via the chain of $\pi\eta\rho\iota$ pointers to enclosing environments.

> The system maintains a unique ___global environment___, whose _pointer-to-enclosing-environment_ is `None`.

> A frame $\phi$ belongs to a virtual sequence of environments implied by the unidirectional pointer-chain of enclosing environments rooted in $\phi$ and ending at the unique global environment. The ___value of a variable in an environment___ is the value in the first binding in any frame of that sequence. Bindings lower in the chain may ___shadow___ bindings higher in the chain, rendering them inaccessible. If no frame in a chain specifies a binding for the variable, then the variable is ___unbound___ in the environment. A variable may be ___bound___ in one environment and unbound in another.

In the Environment class, we override `__getattr__` to avoid a separate function for recursive lookup. We can't also override `__setaddr__` to call `setattr(self.$\phi$ ...)` because `self.$\phi$` diverges on `getattr(self.$\phi$ ...)`.

In [1]:
from dataclasses import dataclass, field
from types import FunctionType
from typing import Any
@dataclass
class Environment:
    """Set attributes via settattr(env.ϕ, key, val). When getting
    attributes, it's ok to omit the ϕ because of overloaded 
    __getattr__."""
    ϕ: FunctionType   # "frame," a nice place to hang attributes
    π: "Environment"  # via Greek πηρι, short name for 'enclosing'
    def _get_it(self, key: str) -> Any:
        "recursive lookup"
        try:
            ρ = getattr(self.ϕ, key)
        except AttributeError as _:
            if self.π is None:
                raise NameError(f'Environment: Name {key} is unbound.')
            else:  # recurse
                ρ = self.π.__getattr__(key)
        return ρ
    def __getattr__(self, key: str) -> Any:
        """recursive lookup by dot notation"""
        return self._get_it(key)
    def __getitem__(self, key: str) -> Any:
        """recursive lookup by bracket notation"""
        return self._get_it(key)
# diverges because it calls __getattr__ for 'self.ϕ'
#    def __setattr__(self, key, val):
#        setattr(getattr(self, 'ϕ'), key, val)
#        setattr(self.ϕ, key, val)
# The ugliness of 'setattr' is hidden in DEFINE.    

# Unique Global Environment $\Gamma\Pi$<a id="global-environment"></a>

The unique global environment is $\Gamma\Pi$, defined once for each session.

In [2]:
ΓΠ = Environment(lambda: None, None)  # Γ for "global," Π for "environment"
setattr(ΓΠ.ϕ, 'γόὂ', 43)
ΓΠ.γόὂ

43

# Procedure(code{body, params}, $\pi$)<a id="procedure"></a>

From SICP again:

> A ___procedure___ is a pair of code and environment. 

___Code___ is a dictionary of parameter names and a $\lambda$ expression. The $\lambda$, by convention, takes a sigle argument, $\pi$, the environment in which its formal parameters are bound to actual arguments via procedure application, as described in SICP 3.2. This convention seems to be the best we can do for composable $\lambda$s in Schemulator. 

For now, we support only positional arguments, one-to-one with the argument list. That's consistent with [Gambit Scheme](https://github.com/gambit/gambit), which reports "Wrong number of arguments ..." if the application has too many or too few arguments.

By default, procedures are bound in the unique global environment, $\Gamma\Pi$.

A `Procedure` includes a `__call__` override for conveniently calling Procedures in their given environments.

In [3]:
from typing import Dict, List, Tuple, Any
Parameters = List[str]  # positional arguments only
class Procedure: 
    """forward reference; will be corrected. Needed to
    spec APPLY."""
    pass
def APPLY(proc: Procedure, 
          args: List[Any], 
          π: Environment = ΓΠ) -> Any:
    """forward reference; will be corrected. Needed to
    spec Procedure."""
    print(args)
@dataclass
class Procedure:
    """Include __call__ override for convenient syntax."""
    code: Dict  # TODO: Check for duplicated symbols in `parameters`.
    π: Environment=ΓΠ  # bound in global environment by default
    def __call__(self, *args):
        return APPLY(self, args, self.π)

Following the example for _square_ in SICP 3.2.1, let's define it in the global environment and test the invocation of `APPLY`, which, for now, just prints its arguments.

In [4]:
setattr(
    ΓΠ.ϕ, 
    "square", 
    Procedure(
        {"body": lambda π: π.x * π.x,  # ugly, I know; sorry :(
         "parameters": ['x']}))
ΓΠ.square(5)
ΓΠ.square(5, 6)

(5,)
(5, 6)


## $\Lambda$($\lambda$, params, $\pi$)

Some syntactical help for anonymous procedures. Default parameter list is empty and default environment is the unique global environment $\Gamma{}\Pi$.

In [5]:
def Λ(body: FunctionType, 
      parameters: List[str] = [],  # default empty
      π = ΓΠ) -> Procedure:        # default global
    ρ = Procedure(
        code={"body": body, "parameters": parameters}, 
        π=π)
    return ρ

Test the $\Lambda$ syntax with the current `APPLY`.

In [6]:
setattr(
    ΓΠ.ϕ,
    "square",
    Λ(lambda π: π.x * π.x, ['x']))
ΓΠ.square(5)
ΓΠ.square(5, 6)

(5,)
(5, 6)


# Var(sym)<a id="var"></a>

Sometimes, we want to interpret strings not as self-evaluating atoms but as references to variables in an environment. We test the type [after defining `EVAL`, below](#eval). `Var` is needed to delay evaluation in [LET_STAR](#let-star) and other constructs.

In [7]:
@dataclass
class Var():
    sym: str

# Application(head, args, $\pi$)<a id="application"></a>

In $\lambda$-calculus, an _Application_ is an unevaluated list with a Procedure or symbol (`str`) in the first slot and unevaluated actual arguments in the remaining positions. This is needed in [LET_STAR](#let-star) and related constructs to delay evaluation until the environment is established, where [EVAL](#eval) cant evaluate them.

`Application` is a placeholder for a more general [QUOTE](#quote) mechanism (TODO).

`Application` includes a `__call__` override for natural calling syntax in parentheses (round brackets).

In [8]:
from typing import Union, Any
class Application:
    """forward reference; corrected immediately below.
    Needed to spec EVAL_APPLICATION"""
    pass
def EVAL_APPLICATION(expr: Application, π: Environment = ΓΠ):
    """forward reference; corrected below"""
    pass
from dataclasses import field
@dataclass
class Application():
    head: Union[str, Procedure]
    args: List[Any] = field(default_factory=list)
    π: Environment = ΓΠ
    def __call__(self):
        EVAL_APPLICATION(self, π)

## $\Xi$(head, args, $\pi$)

Just as [Procedure](#procedure) has a [system-reserved Greek](#greek) shortcut, $\Lambda$, we make a shortcut, $\Xi$, for `Application`.

In [9]:
Ξ = Application

We test this a bit later.

# QUOTE, QUASIQUOTE, UNQUOTE<a id="quote"></a>

TODO

## EVAL(expr, $\pi$)<a id="eval"></a>

`EVAL` calls `APPLY` for Applications. But `APPLY` calls `EVAL` on all arguments, so we must refer to the earlier forward reference for `APPLY` before defining `EVAL`. We test `EVAL` after [`APPLY` is corrected, below](#apply).

First, we correct `EVAL_APPLICATION`. The first slot of an `Application` may contain a string, which is looked up in the given environment, or an explicit procedure. To evaluation an `Application`, evaluate the arguments, then apply the procedure that is either found directly in or looked up from the string in the first slot of the `Application`.

In [10]:
def EVAL_APPLICATION(expr: Application, π: Environment = ΓΠ) -> Any:
    if isinstance(expr.head, str):
        head = EVAL(expr.π[expr.head], expr.π)
        assert isinstance(head, Procedure), \
            f'The head of {expr} must be a string or a Procedure, ' \
            f'not a {expr.head}'
    elif isinstance(expr.head, Procedure): 
        head = expr.head
    else:
        raise ValueError(
            f'The head of {expr} must be a string or a Procedure, '
            f'not a {expr.head}')
    eargs = [EVAL(arg, π) for arg in expr.args]
    ρ = APPLY(head, eargs, π)
    return ρ

Types other than `Application`  or `Var` evaluate to themselves.

In [11]:
from typing import Any, Dict, Tuple, List

def EVAL(expr: Any, π: Environment = ΓΠ) -> Any:
    """Python does a lot of this for us."""
    if isinstance(expr, int) or \
       isinstance(expr, float) or \
       isinstance(expr, str) or \
       isinstance(expr, bool):
        ρ = expr
    elif isinstance(expr, Procedure):
        ρ = expr
    elif isinstance(expr, Dict):
        # self-evaluating, for memo tables:
        ρ = expr
    elif isinstance(expr, Tuple):
        # self-evaluating, for memoized, curried functions
        ρ = expr
    elif isinstance(expr, Var):
        ρ = π[expr.sym]  # recursive lookup in Environment
    elif expr is None:
        ρ = None
    elif isinstance(expr, Application):
        ρ = EVAL_APPLICATION(expr, π)
    else:
        raise ValueError(f'{expr} has unsupported type {type(expr)}')
    return ρ

### Test Var Lookup

We bound the little [Greek](#greek) system-test variable `γόὂ` in [section "Global Environment"](#global-environment).

In [12]:
EVAL(Var('γόὂ'))

43

## APPLY(proc, args, $\pi$)<a id="apply"></a>

By default, procedures are applied in the unique global environment, $\Gamma\Pi$.

Proceudures make a new environment, parented in the given environment, and bind parameters in the new environment to actual arguments evaluated in the old, given environment.

In [13]:
class IllegalArgumentsError(ValueError):
    pass

def APPLY(proc: Procedure, 
          args: List[Any] = [], # default empty argument list
          π: Environment = ΓΠ) -> Any:
    """How about a nice __getitem__ overload on 'Procedure' for this."""
    if len(proc.code['parameters']) != len(args):
        raise IllegalArgumentsError(
            f"Wrong number of arguments passed to procedure {proc}")
    evaled_args = [EVAL(arg, π) for arg in args]
    E1 = Environment(lambda: None, π)
    for k, v in zip(proc.code['parameters'], evaled_args):
        setattr(E1.ϕ, k, v)
    ρ = proc.code['body'](E1)
    return ρ
    
print(APPLY(ΓΠ.square, [5]))

25


Call via "Pythonic" round brackets:

In [14]:
ΓΠ.square(5)

25

Works on anonymous procedures, too:

In [15]:
Λ(lambda π: π.x * π.x, ['x'])(5)

25

Test multiple parameters and arguments:

In [16]:
Λ(lambda π: π.x * π.y, ['x', 'y'])(6, 7)

42

### Test Application

`Application` makes a new environment for any variables.

In [17]:
EVAL(Application(ΓΠ.square, [5]))

25

Test the [Greek](#greek) shortcut for `Application`:

In [18]:
EVAL(Ξ('square', [42]))

1764

`Applications` require `Vars` to help with environments.

In [19]:
EVAL(Ξ('square', [Var('γόὂ')]))

1849

`Applications` may have explicit Procedures in their first slot:

In [27]:
EVAL(Ξ(Λ(lambda π: π.x * π.x, ['x']), [Var('γόὂ')]))

1849

$\Xi$ honors sub-environments:

In [28]:
Λ(lambda π:
  EVAL(Ξ('square', [Var('ϕοοβαρ')]), π),
  ['ϕοοβαρ'])(42)

1764

In [29]:
Λ(lambda π:
  EVAL(Ξ(Λ(lambda π: π.x * π.x, ['x']), 
         [Var('ϕοοβαρ')]), π),
  ['ϕοοβαρ'])(42)

1764

$\phi\omicron\omicron\beta\alpha\rho$ is not bound in the enclosing global environment $\Gamma\Pi$:

In [30]:
try:
    Λ(lambda π:
      EVAL(Ξ('square', [Var('ϕοοβαρ')]), ΓΠ),
      ['ϕοοβαρ'])(42)
except NameError as e:
    print(e.args)

('Environment: Name ϕοοβαρ is unbound.',)


Without `VAR`, $\phi\omicron\omicron\beta\alpha\rho$ is a string. Incidentally, `square` casts the first instance to `Sequence`.

In [31]:
try:
    Λ(lambda π:
      EVAL(Ξ('square', ['ϕοοβαρ']), π),
      ['ϕοοβαρ'])(42)
except TypeError as e:
    print(e.args)

("can't multiply sequence by non-int of type 'str'",)


## DEFINE(sym, val, $\pi$)

Package up the "defining" boilerplate.

By default, `DEFINE` binds symbols in $\Gamma\Pi$.

In [32]:
def DEFINE(sym: str, val: Any, π: Environment=ΓΠ) -> None:
    """official Scheme"""
    setattr(π.ϕ, sym, val)
    return val

DEFINE('saxpy', 
       Λ(lambda π: π.a * π.x + π.y, ['a', 'x', 'y']))

ΓΠ.saxpy(4, 10, 2)

42

### SICP 3.2.2

In [33]:
DEFINE('square', 
       Λ(lambda π: π.x * π.x, ['x']))

DEFINE('sum_of_squares',
       Λ(lambda π: π.square(π.x) + π.square(π.y), ['x', 'y']))

DEFINE('f',
       Λ(lambda π: π.sum_of_squares(1 + π.a, 2 * π.a), ['a']))

ΓΠ.f(5)

136

### SICP Exercise 3.9

In [35]:
DEFINE('factorial',
       Λ(lambda π: 1 if π.n < 2 else \
         π.n * π.factorial(π.n - 1), ['n']))

ΓΠ.factorial(6)

720

This doesn't tail-recurse because Python does not tail-recurse. See [Tail Recursion](#tail-recursion) for mitigation.

In [36]:
DEFINE('fact_iter',
       Λ(lambda π: π.product if π.counter > π.max_count else \
         π.fact_iter(
           π.counter * π.product,
           π.counter + 1,
           π.max_count
           ), ['product', 'counter', 'max_count']))

ΓΠ.fact_iter(1, 1, 6)

720

## Procedures that Apply Procedures

Here is a procedure of `f` and `x` that applies `f` to `x`:

In [37]:
# Outer parens necessary to break lines for comments (Python).
(Λ(lambda π:      # Create environment E1 in ΓΠ
  π.f(π.x),       # Apply E1.f to E1.x.
  ['f', 'x'])     # parameters
(ΓΠ.square, 42))  # <~~~ Bind f to square, x to 42.

1764

### Anonymous Sibling

Here is a procedure that applies an internal procedure; shadowing is no problem, here:

In [38]:
Λ(lambda π:     # Create environment E1 in ΓΠ.
  Λ(lambda π:   # Create environment E2 in ΓΠ.
    π.n * π.n,  # <~~~ n is bound in E2.
    ['n']       #      (E2 is sibling to E1)
   )            # Parent environment implicitly ΓΠ.
  (π.n),        # <~~~ Look up in E1, bind in E2.
  ['n'])(42)    # <~~~ Bind in E1.

1764

### Anonymous Child

Include the non-default $\pi$ on the inner to chain the environments:

In [39]:
Λ(lambda π:     # Create environment E1 in ΓΠ.
  Λ(lambda π:   # Create environment E2 in E1 !!!! 
    π.x * π.n,  # <~~~ n in E1, x in E2.
    ['x'],      #      (E2 is child of E1)
    π)          # Parent environment *explicitly* E1.
  (π.n),        # <~~~ Look up in E1, bind in E2
  ['n'])(42)    # <~~~ Bind in E1

1764

## Procedures that Return Procedures

The $\Lambda$ procedure below is just the identityf function: it returns its argument.

### Known to Parent

In [40]:
Λ(lambda π:  # Create environment E1 in ΓΠ.
  π.f,       # Just return the value of parameter f.
  ['f'])     # Parent environment implicitly ΓΠ.
(ΓΠ.square)  # <~~~ Procedure bound in ΓΠ
(42)         # Apply the returned procedure.

42

### Anonymous

Return a fresh anonymous procedure rather than one bound to a global symbol as above:

In [41]:
Λ(lambda π:                      # Identity function as above.
  π.f, 
  ['f'])
(Λ(lambda π: π.x * π.x, ['x']))  # <~~~ Anonymous procedure
(42)                             # Apply the returned procedure.

42

## $\Upsilon$: Squaring Square Roots of Functions

[See this other noteobook](https://github.com/rebcabin/rebcabin.github.io/blob/main/PythonYCombinators.md). We follow its derivations step-by-step from first principles.

The running example is recursive factorial.

Don't forget non-default $\pi$ on the inner lest `sf` be unbound.

In [42]:
Λ(lambda π: 
  Λ(lambda π: 
    1 if π.n < 1 else π.n * π.sf(π.sf)(π.n - 1), 
    ['n'], π), ['sf'])(
    Λ(lambda π: 
      Λ(lambda π: 
        1 if π.n < 1 else π.n * π.sf(π.sf)(π.n - 1), 
        ['n'], π), ['sf']))(6)

720

Abstract `sf(sf)(m)` into a delayed $\lambda$ of `m`:

In [43]:
Λ(lambda π: 
  Λ(lambda π: 
    Λ(lambda π: 
      1 if π.n < 1 else π.n * π.f(π.n - 1), 
      ['n'], π), 
    ['f'], π)(Λ(lambda π: π.sf(π.sf)(π.m), ['m'], π)), 
  ['sf'])(
Λ(lambda π: 
  Λ(lambda π: 
    Λ(lambda π: 
      1 if π.n < 1 else π.n * π.f(π.n - 1), 
      ['n'], π), 
    ['f'], π)(Λ(lambda π: π.sf(π.sf)(π.m), ['m'], π)), 
  ['sf']))(6)

720

Abstract the _domain code_ into `d`, a function of `f`, the _business code_, is a function of `n`, the _business parameter_.

In [44]:
Λ(lambda π: # of d
  Λ(lambda π: π.d(Λ(lambda π: π.sf(π.sf)(π.m), ['m'], π)),
    ['sf'], π)(
      Λ(lambda π: π.d(Λ(lambda π: π.sf(π.sf)(π.m), ['m'], π)),
        ['sf'], π)), 
  ['d'])(
    Λ(lambda π: # of f
      Λ(lambda π: 
        1 if π.n < 1 else π.n * π.f(π.n - 1), 
        ['n'], π), 
      ['f'])
    )(6)

720

Abstract the self-applied $\lambda$ of `sf` into `g`:

In [45]:
Λ(lambda π: # function of domain code, d
  Λ(lambda π: π.g(π.g), ['g'], π)(
      Λ(lambda π: π.d(Λ(lambda π: π.sf(π.sf)(π.m), ['m'], π)),
        ['sf'], π)), 
  ['d'])(
    Λ(lambda π: # domain code; function of business code, f
      Λ(lambda π: 
        1 if π.n < 1 else π.n * π.f(π.n - 1), # business code
        ['n'], π), # business parameter, n
      ['f']) # recursive function
)(6)

720

## Recursive Factorial via $\Upsilon{}1$

The glyph that looks like "Y" is actually capital Upsilon ($\Upsilon$ in $\LaTeX$). Names in user code should not collide with it if users remember that they should [avoid Greek in user code](#greek).

Package into a system function, $\Upsilon{}1$, for later use. The "1" in the name means that this is for domain codes that return business codes of one parameter:

In [46]:
DEFINE('Υ1', 
       Λ(lambda π: # function of domain code, d
         Λ(lambda π: π.g(π.g), ['g'], π)(
             # of business code of one parameter
             Λ(lambda π: π.d(
                 Λ(lambda π: π.sf(π.sf)(π.m), 
                   ['m'], π)),
               ['sf'], π)), 
         ['d']))

DEFINE('fact_recursive',
      Λ(lambda π: # domain code; function of business code, f
        Λ(lambda π: 
          1 if π.n < 1 else π.n * π.f(π.n - 1), # business code
          ['n'], π), # business parameter, n
        ['f'])) # recursive function

ΓΠ.Υ1(ΓΠ.fact_recursive)(6)

720

## Iterative Factorial via $\Upsilon{}3$

$\Upsilon$ must be tailored for a given number of business parameters. This one is for three. We could write a Python-AST hack to handle any number of business parameters, but that's Python macrology, a rabbit hole to sidestep for now (TODO: reconsider).

In [47]:
# λ d: (λ g: g[g])(λ sf: d[λ m, c, x: sf[sf][m, c, x]]) 
DEFINE('Υ3', 
       Λ(lambda π: # of d, the domain code ...
         Λ(lambda π: π.g(π.g), ['g'], π)(
             # ... of business code of three parameters
             Λ(lambda π: π.d(  # domain code
                 Λ(lambda π: 
                   π.sf(π.sf)(π.m, π.c, π.x),  # business code
                   ['m', 'c', 'x'], π)),  # business parameters
               ['sf'], π)), 
         ['d']));

Here is user-level domain code, redefining `fact_iter` in domain-code form. Any domain code is a function of `f`, recursive _business code_. In this case, `f` is a function of 3 business parameters. This will get us to a tail-recursive solution in the [section on tail recursion](#tail-recursion).

In [48]:
# λ f: λ m, c, x: m if c > x else f(m*c, c+1, x)
DEFINE('fact_iter', # domain code is a function of f ...
       Λ(lambda π: # ... which is business code.
         Λ(lambda π: π.m if π.c > π.x else 
           π.f(π.m * π.c, π.c + 1, π.x), # business code
           ['m', 'c', 'x'], π), # business parameters
         ['f']));

In [49]:
ΓΠ.Υ3(ΓΠ.fact_iter)(1, 1, 6)

720

# Tail Recursion<a id="tail-recursion"></a>

Thanks to [Thomas Baruchel for this idea on tail recursion](https://stackoverflow.com/questions/13591970/does-python-optimize-tail-recursion). 

If users are aware that their domain code is tail-recursive, then they may call it via `LOOP` instead of via $\Upsilon$. In Scheme, detection of tail recursion is automatic. In Python and Schemulator, users must invoke tail recursion explicitly. This isn't terrible. Tail-calls are lexically obvious, so users should always know. In Clojure, there is precedent: users explicitly write `loop` and `recur`. In any event, domain code can always be called via the proper, non-tail-recursive $\Upsilon$, the one that knows the count of business parameters. In our implementation, we imitate Clojure's names `loop` and `recur`. 

`LOOP3` has the same signature as $\Upsilon3$; it takes domain code with business code of three arguments as its sole argument. 

The glyph that looks like "P" below is Greek Capital Rho for "recur." Names in user code will not collide with P if users remember to [avoid Greek](#greek). As with $\Upsilon$, Rho and `LOOP` must know their argument counts. That's OK for now (TODO: reconsider).

In [50]:
class TailCall(Exception):  
    """αναδρομική κλήση"""
    def __init__(self, *args):
        """Overwrite old args with new."""
        self.args = args

def RECUR(*args):  
    """υψώνω: in sincere flattery of Clojure"""
    raise TailCall(*args)

def LOOP3(d: Procedure) -> Procedure:
    """in sincere flattery of Clojure, and thanks to Thomas Baruchel."""
    DEFINE('Ρ3', Λ(lambda π: RECUR(π.m, π.c, π.x), ['m', 'c', 'x']));
    def looper(*args):
        """Expression form of a while-loop statement."""
        while True:
            try: 
                return d(ΓΠ.Ρ3)(*args)
            except TailCall as e:
                args = e.args
    ρ = Λ(lambda π: looper(π.m, π.c, π.x), ['m', 'c', 'x'], π=d.π)
    return ρ

LOOP3(ΓΠ.fact_iter)(1, 1, 20)

2432902008176640000

> ***Results are undefined if you call any `LOOP` function with non-tail-recursive domain code.***

## Prove It on `fact_iter`

The recursive version blows Python's recursion limit.

In [53]:
try:
    print(ΓΠ.Υ3(ΓΠ.fact_iter)(1, 1, 400))
except RecursionError as e:
    print(e.args)

('maximum recursion depth exceeded while calling a Python object',)


The tail-call version does not. Notice the domain code `fact_iter` is EXACTLY the same as in the recursive version above.

In [54]:
try:
    print(LOOP3(ΓΠ.fact_iter)(1, 1, 400))
except RecursionError as e:
    print(e.args)

64034522846623895262347970319503005850702583026002959458684445942802397169186831436278478647463264676294350575035856810848298162883517435228961988646802997937341654150838162426461942352307046244325015114448670890662773914918117331955996440709549671345290477020322434911210797593280795101545372667251627877890009349763765710326350331533965349868386831339352024373788157786791506311858702618270169819740062983025308591298346162272304558339520759611505302236086810433297255194852674432232438669948422404232599805551610635942376961399231917134063858996537970147827206606320217379472010321356624613809077942304597360699567595836096158715129913822286578579549361617654480453222007825818400848436415591229454275384803558374518022675900061399560145595206127211192918105032491008000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


## Tail-Recursive Fibonacci<a id="tail-recursive-fibonacci"></a>

Write domain code for catastropically slow, non-tail-recursive, exponentially diverging Fibonacci:

In [55]:
DEFINE('fib_slow', 
       Λ(lambda π: 
         Λ(lambda π: 1 if π.n < 2 else 
           π.f(π.n - 1) + π.f(π.n - 2), ['n'], π),
         ['f']))

ΓΠ.Υ1(ΓΠ.fib_slow)(6)

13

This is miserable even for $n=23$. You won't want to call it for bigger arguments.

In [56]:
ΓΠ.Υ1(ΓΠ.fib_slow)(23)

46368

The following takes 10 seconds. Uncomment if you want to see the time per iteration: about 1,000 ms; YES, a full second!

In [59]:
# timeit(ΓΠ.Υ1(ΓΠ.fib_slow)(23))

Without linearization, Fibonacci 500 would not complete in $10^{30}$ times the Age of the Universe. One way to linearize is tail recursion. Another way is [memoization](#memoization) (_sic:_ not _memorization_).

Tail-recursive memoization is possible but not necessary. A tail-recursive Fibonacci easy and blazingly fast:

In [60]:
DEFINE('fib_iter',
       Λ(lambda π:
         Λ(lambda π: π.b if π.n < 1 else 
           π.f(π.b, π.a + π.b, π.n - 1),
           ['a', 'b', 'n'], π),
         ['f']));

Check it:

In [61]:
LOOP3(ΓΠ.fib_iter)(0, 1, 23)

46368

Time it:

The following takes 10 seconds. Uncomment if you want see 250 _micro_ seconds, or so, 4000 times faster.

In [64]:
# timeit(LOOP3(ΓΠ.fib_iter)(0, 1, 23))

Stress it, remembering that the non-tail-recursive version would not complete in astronimical time:

In [65]:
LOOP3(ΓΠ.fib_iter)(0, 1, 5000)

6276302800488957086035253108349684055478528702736457439025824448927937256811663264475883711527806250329984690249846819800648580083040107584710332687596562185073640422286799239932615797105974710857095487342820351307477141875012176874307156016229965832589137779724973854362777629878229505500260477136108363709090010421536915488632339240756987974122598603591920306874926755600361865354330444681915154695741851960071089944015319300128574107662757054790648152751366475529121877212785489665101733755898580317984402963873738187000120737824193162011399200547424034440836239726275765901190914513013217132050988064832024783370583789324109052449717186857327239783000020791777804503930439875068662687670678802914269784817022567088069496231111407908953313902398529655056082228598715882365779469902465675715699187225655878240668599547496218159297881601061923195562143932693324644219266564617042934227893371179832389642895285401263875342640468017378925921483580111278055044254198382265567395946431803304304326865077

# Memoized [sic] Fibonacci<a id="memoization"></a>

Fibonacci can be linearized by recording intermediate results in a memo table instead of recomputing them. This is an easy instance of [_Dynamic Programming_](https://en.wikipedia.org/wiki/Dynamic_programming). 

## Curried Memo Table

One way to pass a memo table is through a Curried second argument. We'll need $\Upsilon2C$, generic for 2-parameter, Curried business code:

In [66]:
DEFINE('Υ2C', 
       Λ(lambda π: # function of domain code, d ...
         Λ(lambda π: π.g(π.g), ['g'], π)(
             # with business code of 2 parameters, curried
             Λ(lambda π: 
               π.d(Λ(lambda π: 
                     Λ(lambda π: π.sf(π.sf)(π.m)(π.n), 
                       ['n'], π), ['m'], π)),
               ['sf'], π)), 
         ['d']));

The domain code for a memoized, Curried Fibonacci follows. The parameter `a` is the _accumulator_, _associator_, or memo table, whatever word you like best. This is easiest to read (and to write) from the bottom up. It looks horrendous, but it isn't really.

In [67]:
DEFINE('fib_fast',
       Λ(lambda π: # of f; level 1
         Λ(lambda π: # of a; level 2
           Λ(lambda π: # of n; level 3
             (π.a, 1) if π.n < 2 else
             Λ(lambda π: # of n_1; level 4
               (π.a, π.a[π.n_1]) # optimizer should remove these two lines
               if π.n_1 in π.a else # ^^^
               Λ(lambda π: # of fim1; level 5
                 Λ(lambda π: # of m1; level 6
                   Λ(lambda π: # of r1; level 7
                     Λ(lambda π: # of a1; level 8
                       Λ(lambda π: # of n_2; level 9
                         (π.a1, π.r1 + π.a1[π.n_2]) # <~~~ a quick exit
                         if π.n_2 in π.a1 else 
                         Λ(lambda π: # of fim2; level 10
                           Λ(lambda π: # of m2; level 11
                             Λ(lambda π: # of r2; level 12
                               Λ(lambda π: # of a2; level 13
                                 (π.a2, π.r1 + π.r2), # <~~~ the money line
                                 ['a2'], π)(π.m2[0] | {π.n_2: π.r2}),  # <~~~ update memo
                               ['r2'], π)(π.m2[1]), # unpack
                             ['m2'], π)(π.fim2(π.n_2)), # unpack
                           ['fim2'], π)(π.f(π.a1)), # <~~~ recurse
                         ['n_2'], π)(π.n - 2), # DRY
                       ['a1'], π)(π.m1[0] | {π.n_1: π.r1}), # <~~~ update memo
                     ['r1'], π)(π.m1[1]), # unpack
                   ['m1'], π)(π.fim1(π.n_1)), # unpack
                 ['fim1'], π)(π.f(π.a)), # <~~~ recurse
               ['n_1'], π)(π.n - 1), # DRY
             ['n'], π), # business parameter
           ['a'], π), # curried memo
         ['f'])) # domain code 
ΓΠ.Υ2C(ΓΠ.fib_fast)({})(23)[1]

46368

It's about 1 millisecond per iteration, 1,000 times faster than the original. The following takes 10 seconds. Uncomment if you want proof.

In [71]:
# timeit(ΓΠ.Υ2C(ΓΠ.fib_fast)({})(23)[1])

Still blows the recursion limit:

In [73]:
try:
    print(ΓΠ.Υ2C(ΓΠ.fib_fast)({})(200)[1])
except RecursionError as e:
    print(e.args)

('maximum recursion depth exceeded while calling a Python object',)


## Memo Table as Business Parameter

Before tail-recursion, show the memo as un-Curried. Currying is useful in general, but complicates $\Upsilon$. Get rid of it.

In [74]:
DEFINE('fib_fast_uncurried',
      Λ(lambda π: # of f; level 1
        Λ(lambda π: # of a, n; level 2
          (π.a, 1) if π.n < 2 else
          Λ(lambda π: # of n_1; level 3
            Λ(lambda π: # of t1; level 4
              Λ(lambda π: # of m1; level 5
                Λ(lambda π: # of r1; level 6
                  Λ(lambda π: # of a1; level 7
                    Λ(lambda π: # of n_2; level 8
                      (π.a1, π.r1 + π.a1[π.n_2]) # <~~~ quick exit
                      if π.n_2 in π.a1 else 
                      Λ(lambda π: # of t_2; level 9
                        Λ(lambda π: # of m_2; level 10
                          Λ(lambda π: # of r_2; level 11
                            Λ(lambda π: # of a_2; level 12
                              (π.a2, π.r1 + π.r2), # <~~~ the money line
                              ['a2'], π)(π.m2 | {π.n_2: π.r2}), # <~~~ update memo
                            ['r2'], π)(π.t2[1]), # nupaci
                          ['m2'], π)(π.t2[0]), # unpack
                        ['t2'], π)(π.f(π.a1, π.n_2)), # <~~~ recurse
                      ['n_2'], π)(π.n - 2), # dry
                    ['a1'], π)(π.m1 | {π.n_1: π.r1}), # <~~~ update memo
                  ['r1'], π)(π.t1[1]), # unpac
                ['m1'], π)(π.t1[0]), # unpack
              ['t1'], π)(π.f(π.a, π.n_1)), # <~~~ recurse
            ['n_1'], π)(π.n - 1), # DRY
          ['a', 'n'], π), # busines parameters
        ['f'])); # domain-code signature

Need ordinary $\Upsilon2$ to call it:

In [75]:
DEFINE('Υ2', 
       Λ(lambda π: # of d, the domain code ...
         Λ(lambda π: π.g(π.g), ['g'], π)(
             # of business code of two parameters
             Λ(lambda π: 
               π.d(Λ(lambda π: π.sf(π.sf)(π.m, π.c), 
                     ['m', 'c'], π)), 
               ['sf'], π)), 
         ['d']));

The recursion limit is a little higher, but we don't want any of that.

In [77]:
try:
    print(ΓΠ.Υ2(ΓΠ.fib_fast_uncurried)({}, 200)[1])
except RecursionError as e:
    print(e.args)

453973694165307953197296969697410619233826


## Recursive With Memo

Section [Tail-Recursive Fibonacci](#tail-recursive-fibonacci) exhibits a very short solution without a memo table. Could we write a tail-recursive version with a memo table? Is it worth the effort? Perhaps as a mental exercise.

Pseudocode:

```
f(r1, r2, a, n, x):
    (a, 1) if n < 1 else
    f(r2, r1 + r2, a | {x - (n-1): r1, x - (n-2): r2}, x)
```

### Non-Tail-Recursive

In [78]:
DEFINE('Υ5', 
       Λ(lambda π: # of d, the domain code ...
         Λ(lambda π: π.g(π.g), ['g'], π)(
             # of business code of five parameters
             Λ(lambda π: π.d(
                 Λ(lambda π: π.sf(π.sf)(π.m, π.c, π.x, π.a, π.b), 
                   ['m', 'c', 'x', 'a', 'b'], π)), 
               ['sf'], π)), 
         ['d']));

In [79]:
DEFINE('fib_tc_memo',
      Λ(lambda π: 
        Λ(lambda π:
          (π.a | {π.x: π.r2}, π.r2) if π.n < 1 else \
          π.f(π.r2, π.r1 + π.r2, 
              π.a | {π.x-π.n: π.r2},
              π.n - 1,
              π.x),
         ['r1', 'r2', 'a', 'n', 'x'], π), 
        ['f']));

In [80]:
ΓΠ.Υ5(ΓΠ.fib_tc_memo)(0, 1, {}, 23, 23)

({0: 1,
  1: 1,
  2: 2,
  3: 3,
  4: 5,
  5: 8,
  6: 13,
  7: 21,
  8: 34,
  9: 55,
  10: 89,
  11: 144,
  12: 233,
  13: 377,
  14: 610,
  15: 987,
  16: 1597,
  17: 2584,
  18: 4181,
  19: 6765,
  20: 10946,
  21: 17711,
  22: 28657,
  23: 46368},
 46368)

### Tail-Recusive

In [81]:
def LOOP5(d: Procedure) -> Procedure:
    """in sincere flattery of Clojure, and thanks to Thomas Baruchel."""
    DEFINE('Ρ3', Λ(lambda π: 
                   RECUR(π.m, π.c, π.x, π.a, π.b), 
                   ['m', 'c', 'x', 'a', 'b']));
    def looper(*args):
        """Expression form of a while-loop statement."""
        while True:
            try: 
                return d(ΓΠ.Ρ3)(*args)
            except TailCall as e:
                args = e.args
    ρ = Λ(lambda π: 
               looper(π.m, π.c, π.x, π.a, π.b), 
               ['m', 'c', 'x', 'a', 'b'], π=d.π)
    return ρ

In [82]:
LOOP5(ΓΠ.fib_tc_memo)(0, 1, {}, 23, 23)

({0: 1,
  1: 1,
  2: 2,
  3: 3,
  4: 5,
  5: 8,
  6: 13,
  7: 21,
  8: 34,
  9: 55,
  10: 89,
  11: 144,
  12: 233,
  13: 377,
  14: 610,
  15: 987,
  16: 1597,
  17: 2584,
  18: 4181,
  19: 6765,
  20: 10946,
  21: 17711,
  22: 28657,
  23: 46368},
 46368)

### Test the Limits

In [83]:
try:
    print(ΓΠ.Υ5(ΓΠ.fib_tc_memo)(0, 1, {}, 500, 500)[1])
except RecursionError as e:
    print(e.args)    

('maximum recursion depth exceeded while calling a Python object',)


In [84]:
try:
    print(LOOP5(ΓΠ.fib_tc_memo)(0, 1, {}, 5000, 5000)[1])
except RecursionError as e:
    print(e.args)

6276302800488957086035253108349684055478528702736457439025824448927937256811663264475883711527806250329984690249846819800648580083040107584710332687596562185073640422286799239932615797105974710857095487342820351307477141875012176874307156016229965832589137779724973854362777629878229505500260477136108363709090010421536915488632339240756987974122598603591920306874926755600361865354330444681915154695741851960071089944015319300128574107662757054790648152751366475529121877212785489665101733755898580317984402963873738187000120737824193162011399200547424034440836239726275765901190914513013217132050988064832024783370583789324109052449717186857327239783000020791777804503930439875068662687670678802914269784817022567088069496231111407908953313902398529655056082228598715882365779469902465675715699187225655878240668599547496218159297881601061923195562143932693324644219266564617042934227893371179832389642895285401263875342640468017378925921483580111278055044254198382265567395946431803304304326865077

# DO

```
(DO ((<var1> <init1> <step1>)
     (<var2> <init2> <step2>)
     ...
     (<varN> <initN> <stepN>))
    (<pred> <value>) 
    <optional body>)
```

# SET_BANG

This is assignment without lambda. We do better later, but SICP and _Lambda the Ultimate Imperative_ use it. Implement it to complete coverage of Chapter 3 of SICP.

This has its own recursive lookup, exactly the same as that in [`Procedure`](#procedure), just for a different purpose. 

Tested below in [block](#block). Like Gambit and unlike Common Lisp and Emacs Lisp, we can only `set!` symbols that are already `define`d.

In [85]:
def SET_BANG(
        sym: str, 
        val: Any, 
        π: Environment = ΓΠ) -> None:
    ee = EVAL(val, π)
    """recursive lookup"""
    while π is not None:
        # Find the right π; ...
        try:
            getattr(π.ϕ, sym)  # ... don't recurse via π[sym]
            break
        except AttributeError as _:
            if π.π is None:
                raise NameError(f'Set!: Name {sym} is unbound.')
            else:  # recurse
                π = π.π
    setattr(π.ϕ, sym, ee)
    return None  # following Gambit Scheme

# BLOCK / BEGIN<a id="block"></a>

Sequencing of statements and expressions is not fundamental. Instead, we must chain $\lambda$s. All but the last $\lambda$ are for side-effect. They take one argument and return `None`.

The paper calls the form `BLOCK`. Scheme calls it `BEGIN`. Common Lisp calls it `PROGN`.

Use `_` as a dummy parameter, as does Python.

In [64]:
def BLOCK(*ss: List[Procedure], 
         π: Environment = ΓΠ) -> Any:
    ρ = None
    for s in ss:
        if ρ is not None:
            ρ = APPLY(s, 
                      [APPLY( ρ, [None], π=π)],
                      π=π)
        else:
            ρ = APPLY(s, [None], π)
    return ρ

In [65]:
DEFINE('x', 0)
BLOCK(
    Λ(lambda π: SET_BANG('x', 6, π), ['_']), 
    Λ(lambda π: π.x * 7, ['_']))

42

In [66]:
DEFINE('y', 42)
BLOCK(
    Λ(lambda π: SET_BANG('x', 6, π), ['_']), 
    Λ(lambda π: SET_BANG('x', π.x * 7, π), ['_']),
    Λ(lambda π: π.x * π.y, ['_']))

1764

In [67]:
try:
    BLOCK(
        Λ(lambda π: SET_BANG('x', 6, π), ['_']), 
        Λ(lambda π: SET_BANG('x', π.x * 7, π), ['_']),
        Λ(lambda π: print(π.x * π.y), ['_']),
        Λ(lambda π: π.z, ['_']))
except NameError as e:
    print(e.args)

1764
('Name z is unbound.',)


Test BLOCK in a non-global environment $\pi$:

In [68]:
try:
    Λ(lambda π: 
      print(
      BLOCK(
          Λ(lambda π: SET_BANG('x1', 7, π), ['_'], π),
          Λ(lambda π: SET_BANG('y1', 6, π), ['_'], π),
          Λ(lambda π: π.x1 * π.y1, ['_'], π),
          π=π
      )),
      ['x1', 'y1'])(0, 0)
except NameError as e:
    print(e.args)

42


Names `x1` and `y1` are not in the global environment.

In [69]:
try:
    ΓΠ.x1
except NameError as e:
    print(e.args)
    
try:
    ΓΠ.y1
except NameError as e:
    print(e.args)

('Name x1 is unbound.',)
('Name y1 is unbound.',)


In [70]:
BEGIN = BLOCK

# LET*, LET, LETREC

`LET_STAR` is sequential binding of locals, syntactically like assignment, but purely with $\lambda$ expressions. Later bindings may depend on earlier ones. `LET` is parallel binding, where bindings are independent. `LETREC` is mutually recursive `LET`, where earlier bindings may depend on all other bindings in the form.

Remember that $\Xi$ is a [Greek](#greek) shortcut for [`Application`](#application). Body of any of the _Lets_ must be an `Application` so that the environment $\pi$ can be propagated at the point where it is known.

> ***Results are undefined if the body is not an [`Application`](#application) or $\Xi$.***

## LET*<a id="let-star"></a>

In [71]:
def LET_STAR(
        binding_pairs: List[Tuple[str, Any]], 
        body: Any, 
        π: Environment = ΓΠ) -> Any:
    if len(binding_pairs) == 0:
        ρ = EVAL(body, π)
        return ρ
    key, val = binding_pairs[0]
    if len(binding_pairs) == 1:
        νλ = Λ(lambda π:
              EVAL(body, π),
              [key], π=π)
    else:
        νλ = Λ(lambda π:
              LET_STAR(binding_pairs[1:], body, π),
              [key], π=π)
    ρ = νλ(EVAL(val, π))
    return ρ        

Test depth 0:

In [72]:
LET_STAR([], 
         Ξ(Λ(lambda π: print(43 * 42))))

1806


Test depth 1:

In [73]:
LET_STAR([('z', 42)], 
         Ξ(ΓΠ.square, [Var('z')]))

1764

Test depth 2:

In [74]:
LET_STAR([('z', 42), ('y', 43)], 
         Ξ(Λ(lambda π: print(π.z * π.y))))

1806


Test depth 3, plus dependence on earlier bindings:

In [75]:
LET_STAR([('z', 42), 
          ('y', 43), 
          ('w', Ξ(Λ(lambda π: π.z * π.y, ['z', 'y']), 
                  [Var('z'), Var('y')])
          )], 
         body=Ξ(Λ(lambda π: print(π.w))))

1806


Fully unrolled:

In [76]:
LET_STAR([('z', 42), 
          ('y', Ξ(Λ(lambda π: π.z + 1, ['z']),
                  [Var('z')])), 
          ('w', Ξ(Λ(lambda π: π.z * π.y, ['z', 'y']), 
                  [Var('z'), Var('y')])
          )], 
         body=Ξ(Λ(lambda π: print(π.w))))

1806


## LET<a id="let"></a>

`LET` is parallel "assignment." All variables must be bound in the enclosing environment and may not depend on one another. This implementation is not curried. `body` must be an [`Assignment`](#assignment).

In [77]:
def LET(
        binding_pairs: List[Tuple[str, Any]], 
        body: Any, 
        π: Environment = ΓΠ) -> Any:
    if len(binding_pairs) == 0:
        ρ = EVAL(body, π)
        return ρ
    keys = [pair[0] for pair in binding_pairs]
    vals = [pair[1] for pair in binding_pairs]
    νλ = Λ(lambda π:
          EVAL(body, π),
          keys, π=π)
    ρ = APPLY(νλ, vals, π=π)
    return ρ        

Test depth 0:

In [78]:
LET([], 
    Ξ(Λ(lambda π: print(43 * 42))))

1806


Test depth 1:

In [79]:
LET([('z', 42)], 
    Ξ(ΓΠ.square, [Var('z')]))

1764

Test depth 2:

In [80]:
LET([('z', 42), ('y', 43)], 
    Ξ(Λ(lambda π: print(π.z * π.y))))

1806


Reversed:

In [81]:
LET([('y', 42), ('z', 43)], 
    Ξ(Λ(lambda π: print(π.z * π.y))))

1806


With applications as values, also demonstrating that the inner `y` is evaluated, not lurking in the global environment $\Gamma\Pi$, where `y` is 0:

In [82]:
DEFINE('y', 0)
LET([('y', 42), 
     ('z', Ξ(Λ(lambda π: 42 + 1)))], 
    Ξ(Λ(lambda π: print(π.z * π.y))))

1806


Order does not matter:

In [83]:
LET([('z', Ξ(Λ(lambda π: 42 + 1))),
     ('y', 42)], 
    Ξ(Λ(lambda π: print(π.z * π.y))))

1806


Prove global `y` is unchanged and that `z` is bound only in local environment, not global.

In [84]:
print(ΓΠ.x)
print(ΓΠ.y)
try:
    print(ΓΠ.z)
except NameError as e:
    print(e.args)

42
0
('Name z is unbound.',)


## LETREC

`LETREC` must introduce a new environment to contain the mutually recursive bindings _before_ they're evaluated. `LET` evaluates them in the provided environment before they're bound.

In [85]:
def LETREC(
        binding_pairs: List[Tuple[str, Any]], 
        body: Any, 
        π: Environment = ΓΠ) -> Any:
    if len(binding_pairs) == 0:
        ρ = EVAL(body, π)
        return ρ
    keys = [pair[0] for pair in binding_pairs]
    vals = [pair[1] for pair in binding_pairs]
    E1 = Environment(lambda: None, π)
    _ = [setattr(E1.ϕ, pair[0], pair[1])
         for pair in binding_pairs]
    νλ = Λ(lambda π:
          EVAL(body, π),
          keys, π=π)
    ρ = APPLY(νλ, vals, π=E1)
    return ρ        

Internal recursive references must be EVAL'ed in-situ to access the propagated environment. Otherwise, LETREC is the same as LET

In [86]:
LETREC([('z0', Ξ(Λ(lambda π: 1 + EVAL(π.y0, π)))),
        ('y0', Ξ(Λ(lambda π: 42)))],
       Ξ(Λ(lambda π: π.z0 * π.y0)))

1806

### How to Debug Environments

The following shows the chain of environments, plus it shows a technique for `print`-debugging of environments.

In [87]:
LETREC([('z0', Ξ(Λ(lambda π: 
                   BLOCK(
                       Λ(lambda π: print(π), ['_']),
                       Λ(lambda π: print(ΓΠ), ['_']),
                       Λ(lambda π: 1 + EVAL(π.y0, π), ['_']),
                       π=π),
                       ))),
        ('y0', Ξ(Λ(lambda π: 42)))],
       Ξ(Λ(lambda π: π.z0 * π.y0)))

Environment(φ=<function APPLY.<locals>.<lambda> at 0x105757490>, π=Environment(φ=<function APPLY.<locals>.<lambda> at 0x105788ca0>, π=Environment(φ=<function LETREC.<locals>.<lambda> at 0x105788550>, π=Environment(φ=<function <lambda> at 0x105738550>, π=None))))
Environment(φ=<function <lambda> at 0x105738550>, π=None)


1806

The only difference in the following is `LET` instead of `LETREC`, showing that the required variable `y0` is unbound.

In [88]:
try:
    LET([('z0', Ξ(Λ(lambda π: 
                    BLOCK(
                        Λ(lambda π: print(π), ['_']),
                        Λ(lambda π: print(ΓΠ), ['_']),
                        Λ(lambda π: 1 + EVAL(π.y0, π), ['_']),
                        π=π),
                       ))),
        ('y0', Ξ(Λ(lambda π: 42)))],
       Ξ(Λ(lambda π: π.z0 * π.y0)))
except NameError as e:
    print(e.args)

Environment(φ=<function APPLY.<locals>.<lambda> at 0x105756d40>, π=Environment(φ=<function APPLY.<locals>.<lambda> at 0x105757010>, π=Environment(φ=<function <lambda> at 0x105738550>, π=None)))
Environment(φ=<function <lambda> at 0x105738550>, π=None)
('Name y0 is unbound.',)


The following shows that the bindings of `y0` and `z0` do not leak from `LETREC`:

In [89]:
try:
    print(ΓΠ.z0)
except NameError as e:
    print(e.args)

('Name z0 is unbound.',)


In [90]:
try:
    print(ΓΠ.y0)
except NameError as e:
    print(e.args)

('Name y0 is unbound.',)


# COND

TODO

# Junkyard

Ignore everything below. It's saved in case we need it someday.